# **Projeto final da Disciplina de Processamento de Linguagem Natural: Assistente Virtual do SEI Julgar**

Bem-vindo ao notebook de desenvolvimento do **Assistente Virtual do SEI Julgar**!

O SEI Julgar é um módulo do Sistema Eletrônico de Informações (SEI) que automatiza e gerencia o processo de julgamento de processos administrativos que precisam ir ao colegiado para serem votados.

Este projeto demonstra como construir um chatbot de **Recuperação Aumentada por Geração (RAG)**, especializado em fornecer suporte técnico e instruções detalhadas sobre o uso do módulo **SEI Julgar**.

**Projeto desenvolvido por:**

André Cacau

Vinícius de Souza Sá

---

**Problemas e Justificativa do Trabalho**

Produzimos um manual sobre o módulo SEI Julgar, baseado em manuais de outros órgãos e de documentações oficiais do módulo. Ficou bem extenso e detalhado, o que poderia causar relutância dos usuários em utilizá-lo para dirimir dúvidas ou até mesmo para o entendimento de como executar algum procedimento no sistema. Logo, o chatbot surge para possibilitar um acesso rápido, dinâmico e intuitivo às informações por meio de conversas em linguagem natural.

---
**Objetivo do trabalho**

Chatbot com RAG (*Retrieval-Augmented Generation*) que responde com base no manual do SEI Julgar.


### **Instalação das bibliotecas**

In [ ]:
!pip install -q \
    langchain langchain-groq langchain_community langchain-huggingface \
    faiss-cpu chromadb PyPDF2 streamlit python-dotenv >/dev/null 2>&1

!npm install -g localtunnel >/dev/null 2>&1

!python -m spacy download pt_core_news_md --quiet >/dev/null 2>&1


### **Importe o manual Documentação SEI Julgar - Secretaria.pdf**

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print(f"Arquivo enviado: {filename}")


Saving Documentação SEI Julgar - Secretaria.pdf to Documentação SEI Julgar - Secretaria.pdf
Arquivo enviado: Documentação SEI Julgar - Secretaria.pdf


---
## **Configuração do Large Language Model (LLM)**
Para iniciar o desenvolvimento do chatbot com RAG, definimos e inicializamos o LLM que será responsável por gerar as respostas finais para o usuário.

Neste projeto, utilizamos a Groq Cloud, que se destaca pela sua velocidade de inferência e por ser gratuito.

**Você irá precisa de uma chave Groq para realizar os testes**

Acesse o site oficial:
https://console.groq.com

1. Faça login (Google ou GitHub).

2. No menu lateral esquerdo, clique em API Keys.

3. Clique no botão:
“Create API Key” ou “Generate New Key”

4. Copie a chave gerada.

In [ ]:
import os
from langchain_groq import ChatGroq

id_model = "llama-3.3-70b-versatile"
temperature = 0.7
os.environ["GROQ_API_KEY"] = "coloque sua chave aqui"


### **Carregar o LLM**

In [ ]:
#Cria a instância do modelo LLM que vai gerar as respostas do chatbot.
def load_llm():
    return ChatGroq(
        model=id_model,
        temperature=temperature,
        api_key=os.getenv("GROQ_API_KEY")
    )

llm = load_llm()

# Teste prático mostrando o funcionamento da llm
llm = load_llm()
resposta = llm.invoke("Explique brevemente o que é o SEI Julgar.")
print(" Resposta do modelo:")
print(resposta.content)


 Resposta do modelo:
O SEI Julgar é um sistema eletrônico do Superior Tribunal de Justiça (STJ) que permite a apresentação de recursos e petições eletronicamente, facilitando o acesso à justiça e agilizando os processos judiciais. Ele é uma ferramenta importante para a modernização da justiça no Brasil.


 O teste anterior mostrou que, embora o LLM consiga gerar respostas em linguagem natural, ele não possui conhecimento específico sobre o módulo SEI Julgar.
Isso ocorre porque o modelo foi treinado em um conjunto amplo de textos da internet. Logo:


*   O LLM pode inventar (alucinar).
*   Não tem acesso aos documentos do SEI Julgar.
*   Não consegue responder com precisão perguntas específicas.






---
## **Construção do RAG (Recuperação e Geração)**

Como falamos anteriormente, por padrão o LLM não possui conhecimento do documento do SEI Julgar. Por conta disso, nesse momento que entra o RAG (Retrieval-Augmented Generation).

De forma simples, a LLM busca informações relevantes em uma base de conhecimento antes de formular a resposta.

O RAG funciona em duas grandes etapas:


1.   Recuperação (Retriever)
2.   Geração (RAG Chain)





---
## **Recuperação (Retriever)**
Antes de responder qualquer pergunta, o sistema busca no banco vetorial os trechos mais relevantes do manual.

Essa etapa garante que o modelo receba apenas informações diretamente relacionadas ao SEI Julgar.

Para construir o retriver, iremos prosseguir com as próximas etapas:

### 1. Leitura e preparação dos documentos
Nesta etapa, realizamos a leitura do PDF e extraímos o texto de cada página.

In [ ]:
from PyPDF2 import PdfReader

pdf_file = "Documentação SEI Julgar - Secretaria.pdf"
reader = PdfReader(pdf_file)

texts = []

for page in reader.pages:
    raw = page.extract_text()
    if raw:
        # Remover quebras de linha e múltiplos espaços
        clean = raw.replace("\n", " ")
        clean = " ".join(clean.split())
        texts.append(clean)

print("Páginas lidas:", len(texts))
clean_text = texts[10].replace("\n", " ") # Ler a página 10
clean_text = " ".join(clean_text.split())
print(clean_text[:1000])





Páginas lidas: 45
2.6 - Registrar Presença na Sessão O SEI Julgar permite registrar se cada membro do colegiado está presente ou ausente na Sessão. Esse registro pode ser feito de duas maneiras: ● Registro em grupo: ○ Marque os membros do colegiado utilizando as caixas de seleção (checkbox) na lista. ○ Clique no botão Registrar Presença para confirmar a presença dos selecionados. ○ A presença tem as seguintes modalidades: Presencial ou Remoto ● Registro individual: ○ Utilize o ícone disponível na coluna Ações da tabela Membros do Colegiado para registrar a presença de cada membro separadamente. 9


### 2. Chunking (Segmentação do texto)
Dividimos o texto em partes menores, para que a busca vetorial funcione com maior precisão.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.create_documents(texts)

print("Total de chunks:", len(chunks))
print(chunks[9].page_content[:300])
# Como o pdf vem com muitas quebras de linhas o PyPDF2 coloca \n em muita coisa.


Total de chunks: 99
2. SECRETARIA DO COLEGIADO 2.1 - Criar Sessão de Julgamento No menu lateral do SEI, acesse a opção “Sessões de Julgamento”. Nessa página estarão disponíveis os botões Pesquisar , Nova e Excluir . Para cadastrar uma nova sessão: 1. Clique em “Nova”; 2. Selecione o colegiado responsável; 3. Defina a d


### 3. Criação dos embeddings
Cada chunk é convertido em um vetor numérico que representa seu significado semântico.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
# Teste com um modelo menor, diferente do utilizado na versão final do chatbot
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")


vector_test = embeddings.embed_query("Como criar uma Sessão de Julgamento no SEI Julgar?")
print("Tamanho do vetor de embedding:", len(vector_test))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Tamanho do vetor de embedding: 384


### 4. Construção da base vetorial (ChromaDB)

Armazenamos os embeddings dos chunks em uma base vetorial persistente.

In [ ]:
from langchain_community.vectorstores import Chroma
import os

persist_dir = "./chroma_teste"
os.makedirs(persist_dir, exist_ok=True)

vectorstore = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=persist_dir)

print("Banco vetorial criado com sucesso!")


Banco vetorial criado com sucesso!


### 5. Testando o Recuperador (MMR)

O recuperador MMR seleciona trechos relevantes evitando redundância. Ele equilibra diversidade e relevância, retornando apenas os melhores chunks para formar a resposta do modelo.


In [ ]:
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 3, 'fetch_k': 4, 'lambda_mult': 0.7}
)

query = "Como iniciar uma votação da Sessão de Julgamento?"
docs = retriever.get_relevant_documents(query)

for i, d in enumerate(docs):
    print(f"--- Documento {i+1} ---")
    print(d.page_content[:400])
    print()


--- Documento 1 ---
3.10 - Sessão de Julgamento Aberta O início da votação é condicionado a duas situações: 1. O Desembargador Relator deve disponibilizar o documento da sessão de julgamento juntamente com o provimento respectivo. 2. A presença do Desembargador Relator e do Presidente da Sessão de Julgamento é obrigatória. O provimento e complemento são indicados ao passar o mouse sobre o ícone . Quando o documento é

--- Documento 2 ---
Uma vez que a votação se encerra e a Secretaria do Colegiado finaliza a sessão, uma Certidão do Julgamento é anexada a todos os processos. O processo administrativo, por sua vez, permanece na unidade que o pautou, ao mesmo tempo que é automaticamente encaminhado à unidade do membro do colegiado que proferiu o voto vencedor . 2.8 - Sessão de Julgamento - Suspensa Somente a unidade da Secretaria pos

--- Documento 3 ---
edição são a inclusão de ressalvas aos votos e ajustes no dispositivo que constará da Certidão e da Ata. Após o encerramento da Sessão, 

/tmp/ipython-input-507648491.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


### 6. Função consolidada

Depois de validar cada parte, montamos a função completa `config_retriver` para uso no chatbot.

Leitura  →  Chunking  →  Embeddings  →  Banco vetorial  →  Recuperador.

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader
import os

def config_retriever():


    # Cria o diretório onde o banco vetorial será armazenado
    persist_dir = "./chroma_db"
    os.makedirs(persist_dir, exist_ok=True)

    # 1. Lê o PDF e divide o texto
    pdf_file = "Documentação SEI Julgar - Secretaria.pdf"
    reader = PdfReader(pdf_file)
    texts = [page.extract_text() for page in reader.pages if page.extract_text()]

    # 2.Segmentação (Chunking)
    """
    Chunk_size - Define o tamanho dos chunks
    Chunk_overlap - Define o quanto os chunks se sobrepõem
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.create_documents(texts)

    # 3. Geração de Embeddings
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

    # 4. Cria o banco vetorial com ChromaDB com os embeddings desses chunks
    vectorstore = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=persist_dir)

    """
    O Maximal Marginal Relevance busca no banco os chunks mais relevantes e diversos para evitar a redundância de informações.
    fetch_k - Define a quantidade de chunks que serão recuperados do banco
    k - Define a quantidade de chunks que serão retornados a llm
    lambda_mult - Controla o peso de Relevância e Diversidade 0.7(70% foco na relevância e 30% na diversidade)
    """
    # 5. Recuperador MMR
    return vectorstore.as_retriever(search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 4, 'lambda_mult': 0.7})





---
## **Geração (RAG Chain)**

Depois que o retriever encontra os melhores trechos, o LLM utiliza essas informações para produzir uma resposta final, fiel ao conteúdo do manual.

A seguir iremos ver todo o processo da RAG Chain.

### 1. Prompt para reformular a pergunta (context_prompt)

Antes de consultar o banco vetorial, é importante que o modelo interprete a pergunta atual considerando o histórico da conversa.

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

context_prompt = ChatPromptTemplate.from_messages([
    ("system", "Reformule a pergunta do usuário, não responda."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

#Exemplo
teste = context_prompt.format(
    chat_history=[
        ("human", "Como registrar uma minuta?"),
        ("ai", "Você pode usar a funcionalidade X do SEI Julgar.")
    ],
    input="e para assinar?"
)

print(teste)


System: Reformule a pergunta do usuário, não responda.
Human: Como registrar uma minuta?
AI: Você pode usar a funcionalidade X do SEI Julgar.
Human: e para assinar?


### 2. Criando o History-Aware Retriever

O History-Aware Retriever utiliza o prompt de reformulação para transformar perguntas vagas em consultas claras.

In [ ]:
# Simula o que o history-aware retriever faria internamente:
reformulada = llm.invoke(
    context_prompt.format(
        chat_history=[("human", "Como criar a Sessão de Julgamento?")],
        input="e como eu abro a Sessão?"
    )
)

print("Pergunta reformulada:")
print(reformulada.content)


Pergunta reformulada:
Você gostaria de saber como estabelecer e iniciar uma Sessão de Julgamento, incluindo os passos necessários para criá-la e iniciá-la?


### 3. Prompt principal de QA (qa_prompt)

Este prompt define como o chatbot deve responder.



In [ ]:
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente especialista no SEI Julgar."),
    MessagesPlaceholder("chat_history"),
    ("human", "Pergunta: {input}\n\nContexto do manual:\n{context}")
])

teste = qa_prompt.format(
    chat_history=[("human", "Como registrar uma minuta?")],
    input="Como iniciar uma votação?",
    context="Trecho simulado do manual: Para iniciar uma votação..."
)

print(teste)


System: Você é um assistente especialista no SEI Julgar.
Human: Como registrar uma minuta?
Human: Pergunta: Como iniciar uma votação?

Contexto do manual:
Trecho simulado do manual: Para iniciar uma votação...


### 5. Função consolidada

In [ ]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

#Cria pipeline RAG: reformulação + busca + resposta final.
def config_rag_chain(llm, retriever):

    # Prompt que reformula perguntas do usuário
    context_prompt = ChatPromptTemplate.from_messages([
        ("system", "Reformule perguntas do usuário sem respondê-las."),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])

    # Recuperador com histórico
    history_retriever = create_history_aware_retriever(
        llm=llm, retriever=retriever, prompt=context_prompt
    )

    # Prompt principal de QA
    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        Você é um assistente virtual especializado **exclusivamente** no módulo **SEI Julgar**.
        Forneça respostas **técnicas e objetivas**, baseadas apenas na documentação oficial.

        REGRAS:
        1. Não inventar informações.
        2. Não responder temas fora do SEI Julgar.
        3. Manter tom formal e instrutivo.
        4. Se o tema for irrelevante, diga: "Posso responder apenas sobre o funcionamento do SEI Julgar."
        """),
        MessagesPlaceholder("chat_history"),
        ("human", "Pergunta: {input}\n\nContexto relevante do manual:\n{context}")
    ])

    #envia a llm o prompt acima mais os documentos retornados pelo retriever
    qa_chain = create_stuff_documents_chain(llm, qa_prompt)
    return create_retrieval_chain(history_retriever, qa_chain)

##

---
## **Demonstração (Retriever + RAG)**

Agora que vimos separadamente como funcionam o Retriever e o RAG, vamos fazer um teste simples integrando todas as etapas.

Usuário → Retriever → RAG → Resposta.

#### 1. Criando um documento qualquer




In [ ]:
docs = [
    "O SEI Julgar é um módulo responsável pela condução de sessões de julgamento e votações colegiadas.",

    "Para iniciar a votação de um processo, o membro deve acessar o painel de votação correspondente. "
    "Entretanto, a Secretaria do Colegiado é responsável por liberar a votação para que ela possa ser iniciada.",

    "A Secretaria do Colegiado pode fechar a pauta, o que impede alterações pelos gabinetes dos desembargadores.",

    "É possível acompanhar o andamento da votação na aba Resultados, onde ficam visíveis os votos já registrados.",

    "A votação pode ser encerrada manualmente ou automaticamente conforme as regras configuradas na sessão."
]


#### 2. Criando a base vetorial

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os, shutil

# cria base temporária
persist_dir = "./chroma_demo"
os.makedirs(persist_dir, exist_ok=True)

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
chunks = splitter.create_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory=persist_dir)
retriever_demo = vectorstore.as_retriever()


/tmp/ipython-input-4232430324.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

#### 3. RAG Demo

In [ ]:
rag_demo = config_rag_chain(llm, retriever_demo)


#### 4. Teste final

In [ ]:

# 1 Criar a RAG Chain
rag_chain = config_rag_chain(llm, retriever)


# 2 Pergunta de demonstração
user_question = "Como iniciar uma votação?"

input_text = user_question

# 3 Executa o RAG
response = rag_chain.invoke({
    "input": input_text,
    "chat_history": []
})


# 4 Exibir resultados
print("\n Resposta Final:")
if isinstance(response, dict):
    print(response.get("answer", "Nenhuma resposta encontrada."))

    if "input" in response:
        print("\n Query reformulada:")
        print(response["input"])

    if "context" in response:
        print(f"\n {len(response['context'])} trechos usados:")
        for i, d in enumerate(response["context"][:3], 1):
            content = getattr(d, "page_content", str(d))
            print(f"\n--- Trecho {i} ---\n{content[:350]}...\n")

else:
    print("Retorno inesperado:", response)





 Resposta Final:
Para iniciar uma votação no SEI Julgar, é necessário que haja um documento disponibilizado no processo. Portanto, o primeiro passo é garantir que esse requisito seja atendido. Uma vez que o documento esteja disponível, a votação pode ser iniciada, considerando as funcionalidades e restrições do sistema, como a manipulação dos votos sendo uma função restrita à secretaria.

 Query reformulada:
Como iniciar uma votação?

 3 trechos usados:

--- Trecho 1 ---
uma condição obrigatória que haja um documento disponibilizado no processo para que a votação possa ser iniciada. 6...


--- Trecho 2 ---
para alterar o provimento de um voto já registrado, mesmo o do Relator . A manipulação dos votos da Sessão é uma função restrita à secretaria; todos os outros usuários estão limitados à visualização dos dados. 13...


--- Trecho 3 ---
Desembargador presente. O destaque não remete à votação em si, ou seja, um(a) Desembargador(a) ou a Secretaria pode selecionar um determinado destaque

In [ ]:
# Caso queira limpar a base de teste
shutil.rmtree("./chroma_demo")




Até aqui percorremos todas as etapas essenciais para a construção do chatbot.
Agora, usaremos o Streamlit para demonstrar a versão final do chatbot do SEI Julgar, pois é uma ferramenta eficiente e interativa para desenvolver interfaces web em Python e utilizar CSS personalizado.


### Chave Groq

In [ ]:
%%writefile .env
GROQ_API_KEY=coloque_sua_chave_aqui

Writing .env


### Importe o manual Documentação SEI Julgar (caso não tenha feito anteriormente) e arquivo chatbot_sei.py

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print(f"Arquivo enviado: {filename}")


Saving chatbot_sei.py to chatbot_sei.py
Arquivo enviado: chatbot_sei.py


Após executar o código a seguir, clique no link em azul https://teams-rough-maximize... que irá aparecer no log da execução.

Demora um pouco para base ser criada e o modelo ser carregado, apenas na primeira execução.

In [ ]:
# 1️⃣ Instalar Streamlit e Cloudflared
!pip install streamlit -q
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# 2️⃣ Rodar Streamlit em background
!streamlit run /content/chatbot_sei.py &>/content/logs.txt &

# 3️⃣ Expor via Cloudflared
!cloudflared tunnel --url http://localhost:8501



2025-11-22T22:16:01Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-22T22:16:01Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-22T22:16:04Z INF +--------------------------------------------------------------------------------------------+
2025-11-22T22:16:04Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-22T22:16:04Z INF |  https://medal-numeric-buyers-covering.trycloudflare.c

### Caso queira excluir a base vetorial

In [ ]:
rm -r chroma_db


---
## **Conclusão**
O projeto obteve êxito na criação do Assistente Virtual do SEI Julgar utilizando a arquitetura RAG. Nosso foco principal foi tornar a consulta ao manual muito mais ágil e intuitiva, seguindo as etapas detalhadas neste notebook.

Com essa implementação, garantimos que a Inteligência Artificial forneça respostas técnicas e diretas, baseadas estritamente no manual do SEI Julgar. Isso é essencial para eliminar alucinações por parte da LLM e oferecer a precisão e segurança que as secretarias e gabinetes dos Magistrados necessitam.




